In [ ]:
!pip install vllm

In [ ]:
!pip install -U ipywidgets

In [1]:
base_model_name = 'MTSAIR/Cotype-Nano'
model_name = 'bb_ru_Cotype-Nano_20250123-100550'
lora_adapter = f'/app/models/{model_name}'
merged_name = f'{lora_adapter}-merged'

In [34]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
import torch
import transformers

In [2]:


llm = LLM(model=base_model_name, dtype=torch.bfloat16, trust_remote_code=True, quantization="bitsandbytes", load_format="bitsandbytes", enable_lora=True,qlora_adapter_name_or_path= lora_adapter)
#llm = LLM(model=merged_name, dtype=torch.bfloat16, trust_remote_code=True)
#llm = LLM(model=merged_name, dtype=torch.bfloat16, trust_remote_code=True, quantization="bitsandbytes", load_format="bitsandbytes")

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-25 11:56:22,052	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-25 11:56:29 config.py:510] This model supports multiple tasks: {'classify', 'generate', 'embed', 'score', 'reward'}. Defaulting to 'generate'.
WARNING 01-25 11:56:29 config.py:588] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 01-25 11:56:29 config.py:2056] bitsandbytes quantization is not tested with LoRA yet.
INFO 01-25 11:56:29 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='MTSAIR/Cotype-Nano', speculative_config=None, tokenizer='MTSAIR/Cotype-Nano', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [01:17<00:00, 77.78s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [01:17<00:00, 77.78s/it]



INFO 01-25 11:57:50 model_runner.py:1099] Loading model weights took 1.0966 GB
INFO 01-25 11:57:50 punica_selector.py:11] Using PunicaWrapperGPU.
INFO 01-25 11:57:54 worker.py:241] Memory profiling takes 4.34 seconds
INFO 01-25 11:57:54 worker.py:241] the current vLLM instance can use total_gpu_memory (16.00GiB) x gpu_memory_utilization (0.90) = 14.40GiB
INFO 01-25 11:57:54 worker.py:241] model weights take 1.10GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 11.08GiB.
INFO 01-25 11:57:54 gpu_executor.py:76] # GPU blocks: 25934, # CPU blocks: 9362
INFO 01-25 11:57:54 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 12.66x
INFO 01-25 11:57:55 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occu

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:19<00:00,  1.84it/s]

INFO 01-25 11:58:14 model_runner.py:1535] Graph capturing finished in 19 secs, took 0.00 GiB
INFO 01-25 11:58:14 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 24.00 seconds


In [39]:
system = '''
Ты помощник для работы в системе BlackBox с использованием языка Component Pascal. Твоя задача информативно отвечать на вопросы. 
Ответ необходимо предоставить в формате markdown и выделять код символами ```. 

Пример: 

**Вопрос**: Как реализовать сортировку пузырьком?
**Ответ**: 

Cортировку пузырьком можно реализовать с помощью следующего кода:

```
	PROCEDURE BubbleSort*;  
		VAR i, j: INTEGER; x: Item;
	BEGIN
		FOR i := 1 TO n-1 DO
			FOR j := n-1 TO i BY -1 DO
				IF a[j-1] > a[j] THEN
					x := a[j-1];  a[j-1] := a[j];  a[j] := x
				END
			END
		END
	END BubbleSort;
```
'''

In [41]:
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name)
def prompt_to_chat(prompt):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    if system is not None:
        messages.insert(0, {"role": "system", "content": prompt})
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

In [3]:
lora_request = LoRARequest("lora-test-2", 1, lora_adapter)

/tmp/ipykernel_2531/3163274587.py:1: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request = LoRARequest("lora-test-2", 1, lora_adapter)


In [51]:
# Инференс
sampling_params = SamplingParams(
  min_p= 0.05,
  repetition_penalty= 1.1,
    frequency_penalty=0.0,
  temperature= 0.3,
  top_k= 30,
  top_p= 0.3,
    max_tokens=512)

outputs = llm.generate(prompt_to_chat("Напиши процедуру вывода чисел от 1 до 5 в лог"), sampling_params, lora_request = lora_request)
print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.12s/it, est. speed input: 53.36 toks/s, output: 100.07 toks/s]

Для начала нужно определиться, что такое процедура. Это набор инструкций, который выполняется последовательно. Процедуры могут иметь параметры (например, процедура чтения целого типа), а также возвращать результат выполнения (например, значение переменной). В нашем случае процедура будет содержать только цикл, который будет выполнять операцию перестановки элементов массива.

Сначала напишем процедуру, которая будет принимать массив из пяти элементов, затем вызовем ее, передавая в качестве аргумента массив, содержащий числа от 1 до 5.
После этого мы можем вызвать процедуру BubbleSort, чтобы увидеть, как она работает.

Код процедуры вывода чисел от 1 до 5 в логи:

```
MODULE  LogTest;

	IMPORT  Log := StdLog,  In := i21sysIn;

	PROCEDURE Do*;
		VAR  i: INTEGER;
	BEGIN
		Log.Ln;
		i := 0;
		WHILE  i < 5  DO
			Log.Int( i );  Log.Chacter(" ");  INC(i )
		END;
		Log.Ln
	END Do;

BEGIN
	Do
END LogTest.
```

Запустите эту процедуру, нажав Ctrl+D. Вы должны увидеть в логах следующее:
```
0 1 2

In [31]:
outputs = llm.generate("Как написать процедуру для открытия текстовой view?", sampling_params, lora_request = lora_request)#, lora_request = lora_request
print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.25s/it, est. speed input: 3.05 toks/s, output: 97.54 toks/s]

  Какие параметры будут доступны?

Вот примеры на разных языках программирования:

Для Компонентного Паскаля:
MODULE ObxViews2;
	IMPORT Views, Ports, Stores, Models, Properties;
	
	TYPE View = POINTER TO RECORD (Models.Model) END;
	
	PROCEDURE (v: View) ConnectTo- (m: Models.Model), NEW;
	BEGIN
		v.ConnectTo(m)
	END ConnectTo;

	PROCEDURE Deposit*;
		VAR v: View;
	BEGIN
		NEW(v); Stores.Join(StdInitDir.New(), v);
		Views.Deposit(v)
	END Deposit;

END ObxViews2.

Для Блэкбокса:
MODULE ObxViews2;
	IMPORT Dialog, Views, Ports, Stores, TextModels, TextViews, StdView;
	
	CONST minVersion = 0; maxVersion = 0;
	
	TYPE View = POINTER TO RECORD (TextViews.View) END;
	
	PROCEDURE (v: View) Internalize (VAR rd: Stores.Reader);
		VAR version: INTEGER;
	BEGIN
		rd.ReadVersion(minVersion, maxVersion, version);
		WHILE ~rd.cancelled & (version >= 0) DO
			rd.ReadInt(v.text);
			IF ~rd.cancelled THEN INC(version) END
		END
	END Internalize;

	PROCEDURE (v: View) Externalize (VAR wr: Stores.Writer);
	B